<a href="https://colab.research.google.com/github/LorenzEh/LLM-Vaccine-Hesitency/blob/main/LLM_Data_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preparation
Data will be pulled from my Google Drive. In a subsequent step, the column names will be renamed, based on an excel file, which contains the full variable names. Furthermore, the panel data (collected from the end of March 2020 to the beginning of July 2023) was aggregated. Questions that appeared multiple times across the different waves were consolidated (this was the case for most of the variables). For numerical variables, the mean was calculated, and for categorical variables (the majority of which are ordinal), the mode was used. Undoubtedly, this approach leads to a significant loss of information. However, it offered the following advantages:

1. **Minimizing missing values significantly:** For example, if a categorical variable had 5 missing values for the same person across 35 waves, the mode was simply calculated from the 30 available values.
2. **Greatly simplifying its use as input for an LLM:** Given that this project is just a small showcase I worked on for only a few days, I decided to simplify the data. It would likely be very interesting to feed panel data into an LLM, especially considering the possibility of providing policies in place at the respective time as additional input and contextualizing them with each wave.



In [ ]:
!pip install -q pyreadstat
!pip install -q gdown

In [ ]:
import pandas as pd
import re
import gdown
import numpy as np

In [ ]:
 # download data from my drive
gdown.download(id="1qctiqUK9kS_Hcc-WV-eT1qPW7Ffd832J", output='data.dta', quiet=False) # unfortunatly the csv does not work, therefore we need to continue with the stata format
gdown.download(id="1scCcJr5osfZDycxz-cv9C1YECxyJqFjT", output='desc.xlsx', quiet=False)

data = pd.read_stata("data.dta")
description = pd.read_excel("desc.xlsx")

Downloading...
From: https://drive.google.com/uc?id=1qctiqUK9kS_Hcc-WV-eT1qPW7Ffd832J
To: /content/data.dta
100%|██████████| 31.8M/31.8M [00:00<00:00, 104MB/s]
Downloading...
From: https://drive.google.com/uc?id=1scCcJr5osfZDycxz-cv9C1YECxyJqFjT
To: /content/desc.xlsx
100%|██████████| 471k/471k [00:00<00:00, 85.7MB/s]


In [ ]:
# data.head(10)

In [ ]:
# description.head(10)

In [ ]:
# get variable names
wave_columns = [col for col in description.columns if re.match(r"W\d+-Q-Nr\.", col)]
variable_names = description["Variables"]

print(variable_names)

0                  Block A: Pre-Fragebogen
1                               Geschlecht
2                              Geburtsjahr
3                  Höchster Schulabschluss
4                               Bundesland
                       ...                
1379                              PTV: MFG
1380    Stimmabgabe: Nationalratswahl 2019
1381           Wahlabsicht (Sonntagsfrage)
1382                   Lebenszufriedenheit
1383     Lebenszufriedenheit: Im Jahr 2019
Name: Variables, Length: 1384, dtype: object


In [ ]:
column_map = {} # save the proper variable names in this dict

for i, varname in enumerate(description["Variables"]):
    for wave_col in wave_columns:
        wave_code = description.at[i, wave_col]
        if pd.notna(wave_code):
            # extract the wave number (e.g., 6 from "W6-Q-Nr.")
            wave_number = re.search(r"W(\d+)", wave_col).group(1)
            var_code = wave_code.strip().replace(" ", "")
            full_colname = f"W{wave_number}_{var_code}"

            # Save mapping in the dict
            column_map[full_colname] = varname

# print(column_map)

In [ ]:
# replace the abbrevations with the real variable names
def rename_column(col):
    # match columns
    match = re.match(r"(W\d+)[_]?([A-Z0-9]+)", col)
    # if they match replace the abbrevations with the variable names
    if match:
        wave = match.group(1)
        var_code = match.group(2)
        full_code = f"{wave}_{var_code}"
        if full_code in column_map:
            return f"{wave}_{column_map[full_code]}"
    return col  # leave unchanged if no match

# apply to all columns
data = data.rename(columns=rename_column)

In [ ]:
data.head(10) # looks alright

In [ ]:
# delete unecessary columns, important: weight columns, could be interesting for the "real" project, for now i'm deleting them for simplicity
start_index = data.columns.get_loc("W1W32_WEIGHTP")
data = data.iloc[:, start_index + 1:]

In [ ]:
def aggregate_panel_data(df):
    var_groups = {}

    # group columns by base variable name
    for col in df.columns:
        if '_' in col:
            wave, var = col.split("_", 1)
            var_groups.setdefault(var, []).append(col)
        else:
            var_groups.setdefault(col, []).append(col)

    processed_columns = []

    for var, cols in var_groups.items():

        # check variable type
        first_col = cols[0]

        if pd.api.types.is_numeric_dtype(df[first_col]):
            # numerical variable: take mean
            agg_col = df[cols].mean(axis=1).rename(var)
        else:
            # categorical variable: take mode
            mode_df = df[cols].apply(
                lambda row: row.dropna().mode()[0] if not row.dropna().empty else np.nan,
                axis=1
            ).rename(var)
            agg_col = mode_df.astype(df[first_col].dtype)

        processed_columns.append(agg_col)

    # Concatenate all columns at once
    return pd.concat(processed_columns, axis=1)


agg_data = aggregate_panel_data(data)


In [ ]:
# save data
# from google.colab import files
# agg_data.to_csv('agg_data.csv', index=False)
# files.download('agg_data.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>